<a href="https://colab.research.google.com/github/MK316/workspace/blob/main/ASR01/ASR04_RecRate_SE.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Recognition Rate calculation using nltk: SE data (Synthesized English)

Need a recognition result file and rainbow sentence file

* **SE.zip** (gTTS generated audio files) - completed (asof 0106), [SETR_result.csv](https://raw.githubusercontent.com/MK316/workspace/main/ASR01/results/SETR_result.csv)
* **SK.zip** (gTTS generated audio files) - completed (asof 0106), [SKTR_result.csv](https://raw.githubusercontent.com/MK316/workspace/main/ASR01/results/SKTR_result.csv)
* **SK.zip 2** with medium.en model [SKTR_result_m.csv](https://raw.githubusercontent.com/MK316/workspace/main/ASR01/results/SK_result_m.csv)
* HE.zip (Native production)
* HK.zip (Learners' production)

## Perception rate ~ Error rate

* Perception rate: Percentage of words that match the original text

### Demo

- Sample text: "**The rainbow** is a division of white light into many beautiful colors." (12 words)

- Perceived text: **The lane bowl** is a division of white light into many beautiful colors. (11 words)

Perception rate = (11/12)*100 = 91%  
Error rate = 9%

### Read the rainbow text (19 sentences)

[The rainbow passage sentence file list from github](https://raw.githubusercontent.com/MK316/workspace/main/ASR01/data/rainbow_sentences.csv)

SR result file from github: [SK](https://raw.githubusercontent.com/MK316/workspace/main/ASR01/results/SK_result_0106.csv), [SE](https://raw.githubusercontent.com/MK316/workspace/main/ASR01/results/SE_result_0106.csv)

🌱 Variable: sk = result file after (1) the sentence recognition process (2) Runtime

In [1]:
import pandas as pd

url = "https://raw.githubusercontent.com/MK316/workspace/main/ASR01/data/rainbow_sentences.csv"
rb = pd.read_csv(url)
print(rb.head())

url2 = "https://raw.githubusercontent.com/MK316/workspace/main/ASR01/results/SE_result_0106.csv"
se = pd.read_csv(url2)
se.head()



   SN                                           Sentence
0  S1  When the sunlight strikes raindrops in the air...
1  S2  The rainbow is a division of white light into ...
2  S3  These take the shape of a long round arch, wit...
3  S4  There is, according to legend, a boiling pot o...
4  S5             People look, but no one ever finds it.


,Filename,Runtime,Recognized
0,SE01.wav,0.641,When the sunlight strikes raindrops in the ai...
1,SE02.wav,0.324,The rainbow is a division of white light into...
2,SE03.wav,0.397,These take the shape of a long round arch wit...
3,SE04.wav,0.309,"There is, according to legend, a boiling pot ..."
4,SE05.wav,0.261,"People look, but no one ever finds it."


🌱 Variable: **se**

💾 SER_result.csv (se = [['SN', 'Filename', 'Runtime', 'Sentence', 'Recognized']])

In [2]:
yourdata = [1,3,2]
yourdata[0]
yourdata[1]
yourdata[2]

2

In [3]:
# Add original text to the result file
oritext = list(rb['Sentence'])
sn = list(rb['SN'])

se['Sentence'] = oritext
se['SN'] = sn
se.head()

se = se[['SN', 'Filename', 'Runtime', 'Sentence', 'Recognized']]
cols = list(se.columns.values)
se.to_csv('SER_result.csv', index=False)
se.head()

,SN,Filename,Runtime,Sentence,Recognized
0,S1,SE01.wav,0.641,When the sunlight strikes raindrops in the air...,When the sunlight strikes raindrops in the ai...
1,S2,SE02.wav,0.324,The rainbow is a division of white light into ...,The rainbow is a division of white light into...
2,S3,SE03.wav,0.397,"These take the shape of a long round arch, wit...",These take the shape of a long round arch wit...
3,S4,SE04.wav,0.309,"There is, according to legend, a boiling pot o...","There is, according to legend, a boiling pot ..."
4,S5,SE05.wav,0.261,"People look, but no one ever finds it.","People look, but no one ever finds it."


## Recognition rate and finding mismatching words using NLTK

In [4]:
#@markdown Calculate recognition rate, record missing words => dataframe (SKTR_result.csv) 
se1=se
from nltk.tokenize import RegexpTokenizer

# number of words in the sentence
nws = []
# number of words in the recognized text
nwr = []
# number of missing words 
nmw = []
# number of words actually recognized
nwar = []

# Recgonition Rate
rr = []
#
nr = []
# Missing word list
mw = []
# Correctly recognized word list
recword=[]


for i in range(0,len(se['SN'])):
  t1 = se['Sentence'][i]

# text 1
  txt1 = t1.lower()
  tokenizer = RegexpTokenizer(r'\w+')
  wlist = tokenizer.tokenize(txt1)
  nt = len(wlist)
  nws.append(nt)

# text 2
  t2 = se['Recognized'][i]
  txt2 = t2.lower()
  tokenizer = RegexpTokenizer(r'\w+')
  wlist1 = tokenizer.tokenize(txt2)
  nt1 = len(wlist1)
  nwr.append(nt1)

# Recognition rate

  # from tables.idxutils import calc_chunksize
  # from nltk.downloader import ErrorMessage
# txt1(original text), txt2 (recognized text)
 
  mword = []
  rword = []
  score = 0
  for i in range(0, len(wlist1)):
      w = wlist1[i]

      if w in wlist:
        sc = 1
        rword.append(w)
      else:
        sc = 0
        mword.append(w)

      score = score + sc
      mwords = ', '.join(mword)
      rwords = ', '.join(rword)
  mw.append(mwords)
  missingword = round((score/len(wlist))*100,2)
  nr.append(score)
  recword.append(rwords)

  # RecRate = float(format(missingword, '.2f'))
  # ErrRate = float(format((100.0 - RecRate), '.2f'))
  
  rr.append(missingword)
  # print('Matching words: %d'%score, 'out of %d words'%len(wlist))
  # print("="*50)
  # print('Recognition Rate: %f %%'%RecRate)
  # print('Error Rate: %f %%'%ErrRate)

se1['LenS'] = nws
se1['LenR'] = nwr
se1['N_RecW'] = nr
se1['RecRate'] = rr
se1['Recognized_words'] = recword
se1['MissRec_words'] = mw

se1.head()


,SN,Filename,Runtime,Sentence,Recognized,LenS,LenR,N_RecW,RecRate,Recognized_words,MissRec_words
0,S1,SE01.wav,0.641,When the sunlight strikes raindrops in the air...,When the sunlight strikes raindrops in the ai...,17,17,17,100.0,"when, the, sunlight, strikes, raindrops, in, t...",
1,S2,SE02.wav,0.324,The rainbow is a division of white light into ...,The rainbow is a division of white light into...,12,12,12,100.0,"the, rainbow, is, a, division, of, white, ligh...",
2,S3,SE03.wav,0.397,"These take the shape of a long round arch, wit...",These take the shape of a long round arch wit...,22,22,22,100.0,"these, take, the, shape, of, a, long, round, a...",
3,S4,SE04.wav,0.309,"There is, according to legend, a boiling pot o...","There is, according to legend, a boiling pot ...",13,13,13,100.0,"there, is, according, to, legend, a, boiling, ...",
4,S5,SE05.wav,0.261,"People look, but no one ever finds it.","People look, but no one ever finds it.",8,8,8,100.0,"people, look, but, no, one, ever, finds, it",


💾 Column ordering & save it as csv (e.g., 'SKTR_result_m_0110.csv')

In [5]:
se2 = se1[['SN', 'Filename', 'Runtime', 'Recognized','LenR','Sentence','LenS','N_RecW','RecRate','Recognized_words','MissRec_words']]
cols = list(se2.columns.values)
se2.to_csv('SETR_result_0110.csv', index=False)
se2.head()

,SN,Filename,Runtime,Recognized,LenR,Sentence,LenS,N_RecW,RecRate,Recognized_words,MissRec_words
0,S1,SE01.wav,0.641,When the sunlight strikes raindrops in the ai...,17,When the sunlight strikes raindrops in the air...,17,17,100.0,"when, the, sunlight, strikes, raindrops, in, t...",
1,S2,SE02.wav,0.324,The rainbow is a division of white light into...,12,The rainbow is a division of white light into ...,12,12,100.0,"the, rainbow, is, a, division, of, white, ligh...",
2,S3,SE03.wav,0.397,These take the shape of a long round arch wit...,22,"These take the shape of a long round arch, wit...",22,22,100.0,"these, take, the, shape, of, a, long, round, a...",
3,S4,SE04.wav,0.309,"There is, according to legend, a boiling pot ...",13,"There is, according to legend, a boiling pot o...",13,13,100.0,"there, is, according, to, legend, a, boiling, ...",
4,S5,SE05.wav,0.261,"People look, but no one ever finds it.",8,"People look, but no one ever finds it.",8,8,100.0,"people, look, but, no, one, ever, finds, it",


The end

# 🌀Recognition Rate without stopwords (23.01.10~)

🌱 Variables: 

* se2 = combined results
* se (reduced for nltk process)= sentence, recognized texts

In [6]:
se = se2[['SN','Sentence','Recognized']]
se.head()

,SN,Sentence,Recognized
0,S1,When the sunlight strikes raindrops in the air...,When the sunlight strikes raindrops in the ai...
1,S2,The rainbow is a division of white light into ...,The rainbow is a division of white light into...
2,S3,"These take the shape of a long round arch, wit...",These take the shape of a long round arch wit...
3,S4,"There is, according to legend, a boiling pot o...","There is, according to legend, a boiling pot ..."
4,S5,"People look, but no one ever finds it.","People look, but no one ever finds it."


**Variable: sw = nltk stopword list** (179 words)

💾 stopword list as csv

In [7]:
import nltk
nltk.download('stopwords')
nltk.download('punkt')
from nltk.corpus import stopwords

sw = stopwords.words('english')
print(stopwords.words('english'))
len(sw)

dfs = pd.DataFrame()
dfs['ID'] = range(1,180)
dfs['Stopwords'] = sw
dfs.to_csv('nltkstopwords.csv', index=False)
dfs.head()

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so', 'than', '

,ID,Stopwords
0,1,i
1,2,me
2,3,my
3,4,myself
4,5,we


Variable: sw (stopword list)

In [8]:
# Checking
from nltk.tokenize import word_tokenize
from nltk.tokenize import RegexpTokenizer

tokenizer = RegexpTokenizer(r'\w+')

example_sent = se2['Sentence'][0]
word_tokens = tokenizer.tokenize(example_sent)
stop_words = set(stopwords.words('english'))

filtered_sentence = [w for w in word_tokens if not w.lower() in stop_words]
#with no lower case conversion
filtered_sentence = []
 
for w in word_tokens:
    if w not in stop_words:
        filtered_sentence.append(w)
 
print('Number of word tokens: %d'%len(word_tokens))
print(word_tokens)
print('='*50)
print('Number of words after stopwords: %d'%len(filtered_sentence))
print(filtered_sentence)

Number of word tokens: 17
['When', 'the', 'sunlight', 'strikes', 'raindrops', 'in', 'the', 'air', 'they', 'act', 'as', 'a', 'prism', 'and', 'form', 'a', 'rainbow']
Number of words after stopwords: 9
['When', 'sunlight', 'strikes', 'raindrops', 'air', 'act', 'prism', 'form', 'rainbow']


Variable: se3 = se2 for further process

In [9]:
se3 = se2
se3.head()

,SN,Filename,Runtime,Recognized,LenR,Sentence,LenS,N_RecW,RecRate,Recognized_words,MissRec_words
0,S1,SE01.wav,0.641,When the sunlight strikes raindrops in the ai...,17,When the sunlight strikes raindrops in the air...,17,17,100.0,"when, the, sunlight, strikes, raindrops, in, t...",
1,S2,SE02.wav,0.324,The rainbow is a division of white light into...,12,The rainbow is a division of white light into ...,12,12,100.0,"the, rainbow, is, a, division, of, white, ligh...",
2,S3,SE03.wav,0.397,These take the shape of a long round arch wit...,22,"These take the shape of a long round arch, wit...",22,22,100.0,"these, take, the, shape, of, a, long, round, a...",
3,S4,SE04.wav,0.309,"There is, according to legend, a boiling pot ...",13,"There is, according to legend, a boiling pot o...",13,13,100.0,"there, is, according, to, legend, a, boiling, ...",
4,S5,SE05.wav,0.261,"People look, but no one ever finds it.",8,"People look, but no one ever finds it.",8,8,100.0,"people, look, but, no, one, ever, finds, it",


In [11]:
from pandas.io.formats.style_render import DataFrame
# Bulk process

from nltk.tokenize import word_tokenize
from nltk.tokenize import RegexpTokenizer

tokenizer = RegexpTokenizer(r'\w+')

# list variables to collect each result

sent_wsls = []
rec_wsls = []
rr_ws = []
corecwords = []
newrate = []
# Length of sentence without stopwords, recognized text without stopwords
lenS_sw = []
lenR_sw = []
sn = len(se3['Sentence'])

# 1. removing stopwords in the original & recognized texts
for i in range(0,sn):
  ori_sent = se3['Sentence'][i]
  rec_sent = se3['Recognized'][i]
  word_tokens = tokenizer.tokenize(ori_sent)
  rword_tokens = tokenizer.tokenize(rec_sent)
  stop_words = set(stopwords.words('english'))

  filtered_sentence = [w for w in word_tokens if not w.lower() in stop_words]
  rfiltered_sentence = [w for w in rword_tokens if not w.lower() in stop_words]
  filtered_sentence = []
  rfiltered_sentence = []
 
  for w in word_tokens:
    if w not in stop_words:
       filtered_sentence.append(w)
  fs = ' '.join(filtered_sentence)
  sent_wsls.append(fs)

  for w in rword_tokens:
    if w not in stop_words:
       rfiltered_sentence.append(w)
  rfs = ' '.join(rfiltered_sentence)
  rec_wsls.append(rfs)

  correctwords = []
  score=0
  n1 = len(filtered_sentence)
  n2 = len(rfiltered_sentence)
  for k in range(0,n1):
      if filtered_sentence[k] in rfiltered_sentence:
        sc = 1
        correctwords.append(filtered_sentence[k])
      else:
        sc = 0 
      score = score + sc
  nwrt = round((score/n1)*100,2)
  newrate.append(nwrt)
  rr_ws.append(score)
  cwl = ', '.join(correctwords)
  corecwords.append(cwl)
  lenS_sw.append(n1)
  lenR_sw.append(n2)

#2 removing stopwords in the recognized words


# Fixed due to Copy warning
se3['Sent_ws'] = sent_wsls
se3['LenS_ws'] = lenS_sw
se3['Rec_ws'] = rec_wsls
se3['LenR_ws'] = lenR_sw
se3['Correctwords_ws'] = corecwords
se3['Recwords_ws'] = rr_ws
se3['RecRate_ws'] = newrate

# 3. writing the result in the DataFrame
se3.to_csv('SETR_SW_result.csv')

se3.head()

<ipython-input-11-ef2f1693c459>:69: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  se3['Sent_ws'] = sent_wsls
<ipython-input-11-ef2f1693c459>:70: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  se3['LenS_ws'] = lenS_sw
<ipython-input-11-ef2f1693c459>:71: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#ret

,SN,Filename,Runtime,Recognized,LenR,Sentence,LenS,N_RecW,RecRate,Recognized_words,MissRec_words,Sent_ws,LenS_ws,Rec_ws,LenR_ws,Correctwords_ws,Recwords_ws,RecRate_ws
0,S1,SE01.wav,0.641,When the sunlight strikes raindrops in the ai...,17,When the sunlight strikes raindrops in the air...,17,17,100.0,"when, the, sunlight, strikes, raindrops, in, t...",,When sunlight strikes raindrops air act prism ...,9,When sunlight strikes raindrops air act prism ...,9,"When, sunlight, strikes, raindrops, air, act, ...",9,100.0
1,S2,SE02.wav,0.324,The rainbow is a division of white light into...,12,The rainbow is a division of white light into ...,12,12,100.0,"the, rainbow, is, a, division, of, white, ligh...",,The rainbow division white light many beautifu...,8,The rainbow division white light many beautifu...,8,"The, rainbow, division, white, light, many, be...",8,100.0
2,S3,SE03.wav,0.397,These take the shape of a long round arch wit...,22,"These take the shape of a long round arch, wit...",22,22,100.0,"these, take, the, shape, of, a, long, round, a...",,These take shape long round arch path high two...,13,These take shape long round arch path high two...,13,"These, take, shape, long, round, arch, path, h...",13,100.0
3,S4,SE04.wav,0.309,"There is, according to legend, a boiling pot ...",13,"There is, according to legend, a boiling pot o...",13,13,100.0,"there, is, according, to, legend, a, boiling, ...",,There according legend boiling pot gold one end,8,There according legend boiling pot gold one end,8,"There, according, legend, boiling, pot, gold, ...",8,100.0
4,S5,SE05.wav,0.261,"People look, but no one ever finds it.",8,"People look, but no one ever finds it.",8,8,100.0,"people, look, but, no, one, ever, finds, it",,People look one ever finds,5,People look one ever finds,5,"People, look, one, ever, finds",5,100.0
